# ANALYSIS OF REAL DATA (GSFC)

From real MnKa1, MnKa2 (+other lines) data, it creates a library of optimal filters and reconstruct (and calibrate) data calculating FWHM of lines

**Energy units are (k)eV**

Imports and definitions

PROCESSING

0. Reconstruct data file (singles) with selected record length and library Ka1 and select HR events

1. Read reconstructed events

    1.1. Have a look to events distribution and to variations with baseline and jitter
   
   STUDY OF MnK COMPLEX

2. Jitter correction

    2.1 Plot recon PH vs PHASE (distance between trigger and parabola fit=PHI+LAGS) & Fit a polynomial
    
3. Baseline drift correction

    3.1 Plot jiterr_recon PH vs Baseline & Fit polynomial
    
    3.2 Fit gaussians, create new Gain scale and re-calibrate energies
    
4. Fit histogram of baseline-jitter-corrected energies

    4.1. Test different number of bins and see how residuals respond
    
    4.2. Use the range of number of bins where residuals are quite estable and use them to get different FWHM; then take median value
    
5. Recalibrate energies

6. Plot ALL calibrated energies (No correction for non Mn energies)
    - get new gain scale with fitted line centres
    - recalibrate energies
    - fit again to get FWHM more precisely
    
7. Plot FWHM comparison of different reconstruction methods/lengths    
        

## Imports and definitions

In [2]:
from subprocess import check_call, STDOUT
import os
from astropy.io import fits
import numpy.polynomial.polynomial as poly
from numpy.polynomial import Polynomial as P
import tempfile
from datetime import datetime
import shutil, shlex
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas
from RxLines import RxLines
from fit2GaussAndRatio import fit2GaussAndRatio
from getMaximaDensity import getMaximaDensity
from fit2gauss2hist import fit2gauss2hist
from fit3gauss2hist import fit3gauss2hist
from gainScaleFit import gainScaleLinearFit, gainScalePolyFit
from fitVoigt2hist import fitVoigt2hist
from jitterCorr import jitterCorr
from baseCorr import baseCorr
from applyCorr import applyCorr
from commands import run_comm
from clean_records import remove_invalid_records
from annote import AnnoteFinder
from GSFC import averagePulse, autoDeterminePulseWindowAndThreshold, categorize, deviatonFromAveragePulse,nrecs_larger_than,chi2


import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
from numpy import random
from calibLines import *

import ipywidgets as widgets
%matplotlib widget

cwd = os.getcwd()
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
xmlfileSX = os.environ["SIXTE"] + "/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml"
print(xmlfileSX)

/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml


In [11]:
# data files
channels = (1,3,5,7,9,11,13,15,17,19)
channel = 5
resDir = "channel_" + str(channel)
if not os.path.exists(resDir):
    os.makedirs(resDir)
fileph_singles = resDir + "/pulse_chan" + str(channel) + "_singles.fits" # initial data file with all records and PH_ID column populated
fileph_Kas = resDir + "/pulse_chan" + str(channel) + "_Kas.fits" # data file with only those records with Kas lines (selected by max(ADC))

In [5]:
# select SIRENA parameters for library Kas creation and reconstruction of data files
samprate=195312.5
plen = 8192
oflen = 8000
liblen = 4096 # length of maximum optimal filter for library creation
preBuffer = 1000
pBstr = ""
if preBuffer > 0:
    pBstr = "_pB" + str(preBuffer)
method = "OPTFILT"
F0orB0 = "F0"
nS = 5
sU = 3
sD = 4

suffix = pBstr + "_filt_thS3.5"

In [6]:
# reconstructed files
evt_libKas = resDir + "/" + "evt_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + suffix + ".fits"
evt_libKas_HR = resDir + "/" + "evt_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + suffix + "_HR.fits"
evtKas_libKas = resDir + "/" + "evtKas_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + suffix + ".fits"
evtKas_libKas_HR = resDir + "/" + "evtKas_pulse_chan" + str(channel) + "_libKas_" + "pL" + str(plen) + "_" + method + str(oflen) + suffix + "_HR.fits"

# library Kas
libKas = resDir + "/" + "library_Kas_"+ str(liblen) + suffix + ".fits"

In [7]:
# gain scale
deg = 2 # degree of polynomial for gainscale fit 1=linear; >1: polynomial
lines = (MnKa2_cmass, MnKa1_cmass, MnKb_cmass)

In [8]:
def plotMainXlines(ax, alp_l=0.3, alp_t=0.5):
    """ Plot main X line
    ax: axis of plot
    alp_l: transparency (alpha) of calibration lines markers
    alp_t: transparency (alpha) of calibration lines text
    """
    trans=ax.get_xaxis_transform()
    colors = 3*["green"] + 3*["red"] + 3*["magenta"] + 3*["blue"] + 3*["darkorange"]
    lxx = ["Ka11","Ka21","Kb11"]
    lx = ["Ka1","Ka2","Kb"]
    labels = ["Cr"+ll for ll in lxx] + ["Mn"+ll for ll in lxx] + ["Cu"+ll for ll in lxx] + ["Fe"+ll for ll in lx] + ["Co"+ll for ll in lx]
    xpos   = (5414.874,5405.551,5947.000,  #Cr
              5898.882,5887.772,6490.890,  #Mn
              8047.837,8027.993,8905.532,  #Cu
              6403.840,6390.840,7057.980,  #Fe https://xdb.lbl.gov/Section1/Table_1-2.pdf
              6930.320,6915.300,7649.400)  #Co https://xdb.lbl.gov/Section1/Table_1-2.pdf
    ypos   = np.linspace(0.95,0.05,num=len(colors))
    for i in range(len(colors)):
        if xpos[i] < ax.get_xlim()[0] or xpos[i] > ax.get_xlim()[1]:
            continue
        x=xpos[i]
        ax.axvline(x, color=colors[i], ls=":", alpha=alp_l)
        plt.text(x, ypos[i], labels[i], color=colors[i], transform=trans, alpha=alp_t)

In [9]:
def format_axes(fig):
    for i, ax in enumerate(fig.axes):
        ax.text(0.5, 0.5, "ax%d" % (i+1), va="center", ha="center")
        ax.tick_params(labelbottom=False, labelleft=False)

# PROCESSING

## 0. Reconstruct data file (singles & Kas) with selected record length and select HR events
(try different record lengths to get variation of FWHM)

In [12]:
#%%script false --no-raise-error
#reconstruct ALL events
comm = ("tesreconstruction Recordfile=" + fileph_singles + " TesEventFile=" + evt_libKas + " PulseLength=" + str(plen) +
        " LibraryFile=" + libKas + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) +
        " opmode=1 clobber=yes EnergyMethod=" + method + " XMLFile=" + xmlfileSX + " LbT=0.01" + 
        " filtEeV=" + str(MnKas_cmass) + " OFStrategy=FIXED OFLength=" + str(oflen) + " preBuffer=" + str(preBuffer))
mess = "Reconstructing ALL data w/ library (Kas)"
run_comm(comm,msg=mess)
print("####################################")
print("Reconstruction of ALL pulses finished")
print("####################################")

Reconstructing ALL data w/ library (Kas)
tesreconstruction Recordfile=channel_5/pulse_chan5_singles.fits TesEventFile=channel_5/evt_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5.fits PulseLength=4096 LibraryFile=channel_5/library_Kas_4096_pB1000_filt_thS3.5.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=1 clobber=yes EnergyMethod=OPTFILT XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml LbT=0.01 filtEeV=5894.40 OFStrategy=FIXED OFLength=4096 preBuffer=1000
####################################
Reconstruction of ALL pulses finished
####################################


In [14]:
#%%script false --no-raise-error
#reconstruct Mn Kas events
comm = ("tesreconstruction Recordfile=" + fileph_Kas + " TesEventFile=" + evtKas_libKas + " PulseLength=" + str(plen) +
        " LibraryFile=" + libKas + " samplesUp=" + str(sU) + " nSgms=" + str(nS) + " samplesDown=" + str(sD) +
        " opmode=1 clobber=yes EnergyMethod=" + method + " XMLFile=" + xmlfileSX + " LbT=0.01" + 
        " filtEeV=" + str(MnKas_cmass) + " OFStrategy=FIXED OFLength=" + str(oflen) + " preBuffer=" + str(preBuffer))
mess = "Reconstructing Kas data w/ library (Kas)"
run_comm(comm,msg=mess)
print("####################################")
print("Reconstruction of Kas pulses finished")
print("####################################")

Reconstructing Kas data w/ library (Kas)
tesreconstruction Recordfile=channel_5/pulse_chan5_Kas.fits TesEventFile=channel_5/evtKas_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5.fits PulseLength=4096 LibraryFile=channel_5/library_Kas_4096_pB1000_filt_thS3.5.fits samplesUp=3 nSgms=5 samplesDown=4 opmode=1 clobber=yes EnergyMethod=OPTFILT XMLFile=/home/ceballos/sw/SIXTE/git/gitInstall/share/sixte/instruments/athena-xifu/xifu_detector_lpa_75um_AR0.5_pixoffset_mux40_pitch275um_GSFC.xml LbT=0.01 filtEeV=5894.40 OFStrategy=FIXED OFLength=4096 preBuffer=1000
####################################
Reconstruction of Kas pulses finished
####################################


In [15]:
#%%script false --no-raise-error
# select only HR events (ALL)
comm = ("fselect  infile=" + evt_libKas + " outfile=" + evt_libKas_HR + " expr='GRADE1 >= " + str(min(plen,oflen)) + 
       " && GRADE2 > 500' clobber=yes")
run_comm(comm, msg="Selecting HR evts")

Selecting HR evts
fselect  infile=channel_5/evt_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5.fits outfile=channel_5/evt_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5_HR.fits expr='GRADE1 >= 4096 && GRADE2 > 500' clobber=yes


In [16]:
#%%script false --no-raise-error
# select only HR events (Mn Kas)
comm = ("fselect  infile=" + evtKas_libKas + " outfile=" + evtKas_libKas_HR + " expr='GRADE1 >= " + str(min(plen,oflen)) + 
       " && GRADE2 > 500' clobber=yes")
run_comm(comm, msg="Selecting HR evts")

Selecting HR evts
fselect  infile=channel_5/evtKas_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5.fits outfile=channel_5/evtKas_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5_HR.fits expr='GRADE1 >= 4096 && GRADE2 > 500' clobber=yes


## 1. Read reconstructed (ALL) events

In [17]:
pulseFile = resDir + "/pulse.txt"
colname = "'SIGNAL, PH_ID, GRADE1, GRADE2, PHI, LAGS, BSLN'" 
comm = ("fdump wrap=yes infile=" + evt_libKas_HR + "+1 columns=" + colname + " rows='-' prhead=no " +
        "showcol=yes showunit=no showrow=no outfile=" + pulseFile + " clobber=yes pagewidth=256")
run_comm(comm, msg="FDUMPing evt file")

data_HR = pandas.read_csv(pulseFile, skiprows=0,sep="\s+")
print("\nNumber of initial (all energies) HR pulses:", len(data_HR)) 
os.remove(pulseFile)
#display(data_HR)

# exclude events with PHI=0. It'll mean that there is a pulse (undetected) over the rise of another one
data_HR = data_HR[(data_HR.PHI>0.) | (data_HR.PHI<0)]
print("\nNumber of non-excluded events (all energies) HR pulses (after PHI selection):", len(data_HR)) 

# exclude high baseline events: they have a truncated pulse at the beginnig of the record
data_HR = data_HR[(data_HR.BSLN<8325)]
print("\nNumber of non-excluded events (all energies) HR pulses (after BSLN selection):", len(data_HR)) 

print("MinSIGNAL, MaxSIGNAL=", min(data_HR.SIGNAL), max(data_HR.SIGNAL))
print("MinBSLN, MaxBSLN=", min(data_HR.BSLN), max(data_HR.BSLN))
display(data_HR)


FDUMPing evt file
fdump wrap=yes infile=channel_5/evt_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5_HR.fits+1 columns='SIGNAL, PH_ID, GRADE1, GRADE2, PHI, LAGS, BSLN' rows='-' prhead=no showcol=yes showunit=no showrow=no outfile=channel_5/pulse.txt clobber=yes pagewidth=256

Number of initial (all energies) HR pulses: 45018

Number of non-excluded events (all energies) HR pulses (after PHI selection): 45008

Number of non-excluded events (all energies) HR pulses (after BSLN selection): 45006
MinSIGNAL, MaxSIGNAL= 0.2376232451539499 12.89560871484251
MinBSLN, MaxBSLN= 7746.215053763442 8112.046594982079


,SIGNAL,PH_ID,GRADE1,GRADE2,PHI,LAGS,BSLN
0,5.902065,0,4096,4096,0.496958,0,7815.443932
1,6.360335,1,4096,4096,-0.198424,0,7816.058884
2,5.901323,2,4096,4096,0.036803,0,7813.750640
3,5.895881,3,4096,4096,0.020946,0,7816.303123
4,6.367134,4,4096,4096,0.379146,0,7814.694316
...,...,...,...,...,...,...,...
45013,5.895983,46870,4096,4096,-0.004100,0,7773.057860
45014,5.892729,46871,4096,4096,0.386678,0,7773.845366
45015,5.900808,46872,4096,4096,0.278368,0,7772.790579
45016,5.898329,46873,4096,4096,-0.425834,0,7774.009217


### 1.1 Have a look to (ALL) events distribution and to variations with baseline and jitter

In [18]:
# main lines in reconstructed PH units
PHlines = dict()
PHlines = {"Sc-Ka": (4.250,4.400),
           "Ti-Ka": (4.675,4.750),
           "V-Ka":  (5.050,5.125),
           "Cr-Ka": (5.400,5.600),
           "Mn-Ka": (5.860,5.920),
           #"Mn-Ka": (5.875,5.910),
           "Fe-Ka": (6.250,6.350),
           "Mn-Kb": (6.340,6.400),
           "Co-Ka": (6.630,6.800),
           "Ni-Ka": (7.050,7.240),
           "Cu-Ka": (7.520,7.600),
           "Zn-Ka": (7.920,8.050),
           "Ge-Ka": (8.750,8.860),
           "Br-Ka": (9.970,10.190)
          }


In [19]:
#%%script false --no-raise-error

# Events PH distribution
plt.close()
fig = plt.figure(constrained_layout=True, figsize=(10,8))
gs = GridSpec(2, 2, figure=fig)

# Have a look to histogram of ALL reconstructed data
ax1 = fig.add_subplot(gs[0, :])
npulses = len(data_HR.SIGNAL)
ax1.set_xlabel("Reconstructed PH (ma.u.)")
ax1.set_ylabel("#events")
ax1.set_title(("Distribution of ALL events (" + str(npulses) + ")"))
ax1.hist(1e3*data_HR.SIGNAL, bins=2000, density=False, label="Histogram", alpha=0.5, log=True)
ax1.set_xlim(2000,11000)
for line in PHlines:
    PHmin,PHmax = PHlines[line]
    if line == "Mn-Ka":
        ax1.text(1e3*PHmin, 4e3, line)
    else:
        ax1.text(1e3*PHmin, 3e3*random.uniform(0.1,0.7), line)

# Have a look to histogram of Mn Kas
ax2 = fig.add_subplot(gs[1, 0])
minPH,maxPH = PHlines["Mn-Ka"]
npulses = len(data_HR[(data_HR.SIGNAL>minPH) & (data_HR.SIGNAL<maxPH)].SIGNAL)
ax2.hist(1e3*data_HR[(data_HR.SIGNAL>minPH) & (data_HR.SIGNAL<maxPH)].SIGNAL, bins=100, density=False, label="Histogram", alpha=0.5, color="tab:red")
ax2.set_xlabel("Reconstructed PH (a.u.)")
ax2.set_ylabel("# events")
ax2.set_title(("Distribution of Mn Kas events (" + str(npulses) + ")"))
ax2.set_xlim(1e3*minPH,1e3*maxPH)
#ax3.set_xlim(8650,8900) # 0-pad 2048
#ax3.set_xlim(9750,9900) # 0-pad 1024

# Have a look to histogram of Kb
ax3 = fig.add_subplot(gs[1, 1])
(minPH,maxPH) = PHlines["Mn-Kb"]
#(minKb,maxKb) = (9.210, 9.310) # 0-pad 2048
#(minKb,maxKb) = (10.275, 10.350) # 0-pad
npulses = len(data_HR[(data_HR.SIGNAL>minPH) & (data_HR.SIGNAL<maxPH)].SIGNAL)
ax3.hist(1e3*data_HR[(data_HR.SIGNAL>minPH) & (data_HR.SIGNAL<maxPH)].SIGNAL, bins=100, density=False, label="Histogram", alpha=0.5, color="tab:red")
ax3.set_xlabel("Reconstructed PH (a.u.)")
ax3.set_ylabel("# events")
ax3.set_title(("Distribution of Mn Kb events (" + str(npulses) + ")"))
ax3.set_xlim(1e3*minPH,1e3*maxPH)
#ax4.set_xlim(9210,9310) # 0-pad 2048
#ax4.set_xlim(10275,10350) # 0-pad 1024



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(6340.0, 6400.0)

In [20]:
fig = plt.figure(figsize=(10,10))
i = 0
for line in PHlines:
    i+=1
    color="C"+str(i)
    #print("Plotting line=", PHlines[line])
    PHmin,PHmax = PHlines[line]
    npulses = len(data_HR[(data_HR.SIGNAL>PHmin) & (data_HR.SIGNAL<PHmax)].SIGNAL)
    ax = fig.add_subplot(4, 4, i)
    dataLine = data_HR[(data_HR.SIGNAL>PHmin) & (data_HR.SIGNAL<PHmax)].SIGNAL
    ax.hist(1e3*dataLine, bins=80, density=False, alpha=0.5, color=color, label=line)
    ax.set_xlabel("Reconstructed PH (a.u.)")
    ax.set_ylabel("# events")
    ax.set_xlim(1e3*PHmin,1e3*PHmax)
    ax.legend()

fig.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
# plot jitters
plt.close()
phase_HR = data_HR.PHI + data_HR.LAGS
base_HR = data_HR.BSLN

fig = plt.figure(figsize=(11,10))
i = 0
for line in PHlines:
    i+=1
    #print("Plotting line=", PHlines[line])
    PHmin,PHmax = PHlines[line]
    dataLine = data_HR[(data_HR.SIGNAL>PHmin) & (data_HR.SIGNAL<PHmax)]
    dataLine_norm = dataLine.SIGNAL/(np.median(dataLine.SIGNAL))
    phase_HR = dataLine.PHI + dataLine.LAGS
    phase_HR = dataLine.PHI 
    ax1 = fig.add_subplot(4, 4, i)
    ax1.set_xlabel("Phase (samples)")
    ax1.set_ylabel("Reconstructed PH (a.u.)")
    title = "Distribution of " + line + " events"
    ax1.set_title(title)
    ax1.scatter(phase_HR, dataLine_norm, alpha=0.5,s=5)
    #ax1.scatter(phase_HR, dataLine.LAGS, alpha=0.5)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# Jitter and baseline distribution
plt.close()
phase_HR = data_HR.PHI + data_HR.LAGS
base_HR = data_HR.BSLN

fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_xlabel("Phase (samples)")
ax1.set_ylabel("Reconstructed PH (a.u.)")
ax1.set_title("Distribution of ALL events")
ax1.scatter(phase_HR, data_HR.SIGNAL, alpha=0.5)

ax2 = fig.add_subplot(1, 2, 2)
ax2.set_xlabel("Baseline (ADC a.u.)")
ax2.set_ylabel("Reconstructed PH (a.u.)")
ax2.set_title("Distribution of ALL events")
ax2.scatter(base_HR, data_HR.SIGNAL, alpha=0.1)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# STUDY Mn K complex 

## 1. Read reconstructed (Kas) events

In [23]:
pulseFile = resDir + "/pulse.txt"
colname = "'TIME,SIGNAL, PH_ID, GRADE1, GRADE2, PHI, LAGS, BSLN'" 
comm = ("fdump wrap=yes infile=" + evtKas_libKas_HR + "+1 columns=" + colname + " rows='-' prhead=no " +
        "showcol=yes showunit=no showrow=no outfile=" + pulseFile + " clobber=yes pagewidth=256")

run_comm(comm, msg="FDUMPing evt file")

dataKas_HR = pandas.read_csv(pulseFile, skiprows=0,sep="\s+")
print("\nNumber of initial (all energies) HR pulses:", len(data_HR)) 
os.remove(pulseFile)
if len(dataKas_HR[dataKas_HR.PHI == 0.].PHI >0):
    print("Warning: some dataKas have PHI==0. => double undetected pulse?")

display(dataKas_HR)


FDUMPing evt file
fdump wrap=yes infile=channel_5/evtKas_pulse_chan5_libKas_pL4096_OPTFILT4096_pB1000_filt_thS3.5_HR.fits+1 columns='TIME,SIGNAL, PH_ID, GRADE1, GRADE2, PHI, LAGS, BSLN' rows='-' prhead=no showcol=yes showunit=no showrow=no outfile=channel_5/pulse.txt clobber=yes pagewidth=256

Number of initial (all energies) HR pulses: 45006


,TIME,SIGNAL,PH_ID,GRADE1,GRADE2,PHI,LAGS,BSLN
0,1.578693e+09,5.902065,0,4096,4096,0.496958,0,7815.443932
1,1.578693e+09,5.901323,2,4096,4096,0.036803,0,7813.750640
2,1.578693e+09,5.895881,3,4096,4096,0.020946,0,7816.303123
3,1.578693e+09,5.896215,7,4096,4096,0.190314,0,7814.715310
4,1.578693e+09,5.904349,8,4096,4096,0.277119,0,7814.933948
...,...,...,...,...,...,...,...,...
9639,1.578743e+09,5.890862,46869,4096,4096,0.402707,0,7769.918587
9640,1.578743e+09,5.895983,46870,4096,4096,-0.004100,0,7773.057860
9641,1.578743e+09,5.892729,46871,4096,4096,0.386678,0,7773.845366
9642,1.578743e+09,5.900808,46872,4096,4096,0.278368,0,7772.790579


## 0. Remove XT from files in same TDM column

In [24]:
sepTH = plen/samprate #(s)
t1 = preBuffer/samprate #(s) before pulse
t2 = (plen-preBuffer)/samprate #(s) after pulse
time_window = [t1, t2]
dataKas_noXT = dataKas_HR.copy() # inititalize non-XT events

In [25]:
print("Starting with", len(dataKas_noXT.TIME), "events in channel", channel)
for ich in channels:
    print ("Analysing events in channel", ich)
    if ich==channel:
        continue
    fileXT = "evt_pulse_chan" + str(ich) + ".fits" #Full events file in channel ich
    f = fits.open(fileXT)
    TIMExt = f["EVENTS"].data['TIME']
    PHIDxt = f["EVENTS"].data['PH_ID']
    f.close()
    # look for events in fileXT synchronous with evts in fileph_Kas    
    
    xt0 = len(dataKas_noXT.TIME)
    for i in range(len(TIMExt)):
        #dataKas_noXT = dataKas_noXT[(abs(dataKas_noXT.TIME-TIMExt[i])>sepTH)]
        dataKas_noXT = dataKas_noXT[((dataKas_noXT.TIME-TIMExt[i])>t1) | 
                                   ((TIMExt[i]-dataKas_noXT.TIME)>t2)]
        #dataKas_XT = dataKas_HR[(abs(dataKas_HR.TIME-TIMExt[i])<sepTH)]
        #if len(dataKas_XT.TIME >0):
        #    display(dataKas_HR[(abs(dataKas_HR.TIME-TIMExt[i])<sepTH)])
        #time_dist = np.abs(np.array(dataKas_HR.TIME)-TIMExt[i])
        #idx_close = np.argwhere(time_dist < sepTH)[:,0]
        #if len(idx_close)>0:
        #    print("XT photon", i, "in channel", ich, "is very close to Kas photon(s) in df row:",idx_close)
    xt1 = len(dataKas_noXT.TIME)    
    if xt1 < xt0:
        diff=xt0-xt1
        print("Removed",diff,"photons from channel",channel," which have channel-",ich,"XT photons in pulse length")
        

Starting with 9644 events in channel 5
Analysing events in channel 1
Removed 302 photons from channel 5  which have channel- 1 XT photons in pulse length
Analysing events in channel 3
Removed 134 photons from channel 5  which have channel- 3 XT photons in pulse length
Analysing events in channel 5
Analysing events in channel 7
Analysing events in channel 9
Removed 292 photons from channel 5  which have channel- 9 XT photons in pulse length
Analysing events in channel 11
Removed 274 photons from channel 5  which have channel- 11 XT photons in pulse length
Analysing events in channel 13
Removed 214 photons from channel 5  which have channel- 13 XT photons in pulse length
Analysing events in channel 15
Removed 268 photons from channel 5  which have channel- 15 XT photons in pulse length
Analysing events in channel 17
Analysing events in channel 19


In [26]:
rem = len(dataKas_HR.TIME) - len(dataKas_noXT.TIME)
print("Removed", rem, "photons from channel", channel, "list. Ramining photons:", len(dataKas_noXT.TIME))
#display(dataKas_noXT)
dataKas_noXT[650:651]

Removed 1484 photons from channel 5 list. Ramining photons: 8160


,TIME,SIGNAL,PH_ID,GRADE1,GRADE2,PHI,LAGS,BSLN
756,1.578694e+09,5.903252,1092,4096,4096,0.460095,0,7795.97491


In [27]:
dataKas_HR[650:655]

,TIME,SIGNAL,PH_ID,GRADE1,GRADE2,PHI,LAGS,BSLN
650,1.578694e+09,5.904641,951,4096,4096,-0.007028,0,7800.628264
651,1.578694e+09,5.893541,952,4096,4096,0.428414,0,7797.896057
652,1.578694e+09,5.900550,954,4096,4096,0.470910,0,7800.316948
653,1.578694e+09,5.901136,956,4096,4096,-0.381722,0,7800.306196
654,1.578694e+09,5.895271,957,4096,4096,0.308215,0,7797.491551


In [28]:
noXT_PHID=dataKas_noXT.PH_ID

## 2. Jitter correction

### 2.1 Plot Pulse Height vs PHASE (distance between trigger and parabola fit = PHI + LAGS) & fit polynomial

In [29]:
plt.close()
fig = plt.figure(figsize=(9, 7))
ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
ax4 = fig.add_subplot(2,2,4)
# 4096
PHminKas, PHmaxKas = PHlines["Mn-Ka"] 
PHminKb, PHmaxKb = PHlines["Mn-Kb"]

# dataKas_HR already read above
dataKb_HR = data_HR[(data_HR.SIGNAL>PHminKb) & (data_HR.SIGNAL<PHmaxKb)]

print("Number of pulses in dataKas_HR: ", len(dataKas_HR))
#phaseKas_HR = dataKas_HR.PHI + dataKas_HR.LAGS
phaseKas_HR = dataKas_noXT.PHI + dataKas_noXT.LAGS
#dataKas_HR_jitter = applyCorr(ydata=dataKas_HR.SIGNAL, xdata=phaseKas_HR, ax0=ax1, ax1=ax2, alpha=0.5)
dataKas_HR_jitter = applyCorr(ydata=dataKas_noXT.SIGNAL, xdata=phaseKas_HR, ax0=ax1, ax1=ax2, alpha=0.5)
ax1.set_xlabel("Phase (samples)")
ax1.set_ylabel("PH (a.u.)")
ax2.set_xlabel("Phase (samples)")
ax2.set_ylabel("Corrected PH (a.u.)")

print("\nNumber of pulses in dataKb_HR: ", len(dataKb_HR))
phaseKb_HR = dataKb_HR.PHI + dataKb_HR.LAGS
dataKb_HR_jitter = applyCorr(ydata=dataKb_HR.SIGNAL, xdata=phaseKb_HR, ax0=ax3, ax1=ax4, alpha=0.5)
ax3.set_xlabel("Phase (samples)")
ax3.set_ylabel("PH (a.u.)")
ax4.set_xlabel("Phase (samples)")
ax4.set_ylabel("Corrected PH (a.u.)")

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Number of pulses in dataKas_HR:  9644
Fit Y=5.894e+00+ (-2.990e-04)*x**1+ (4.462e-03)*x**2
Fit Y corrected=5.894e+00+ (1.661e-15)*x**1+ (-1.315e-16)*x**2

Number of pulses in dataKb_HR:  1563
Fit Y=6.364e+00+ (-1.797e-03)*x**1+ (7.610e-03)*x**2
Fit Y corrected=6.364e+00+ (3.965e-15)*x**1+ (1.536e-15)*x**2


## 3. Baseline correction

### Plot jiterr_recon PH vs Baseline & Fit polynomial


In [30]:
plt.close()
fig = plt.figure(figsize=(9, 4))
ax0 = fig.add_subplot(1,2,1)
ax1 = fig.add_subplot(1,2,2)

print("Mn Kas")
#baseKas_HR = dataKas_HR.BSLN
baseKas_HR = dataKas_noXT.BSLN
dataKas_HR_jitter_bsln = applyCorr(ydata=dataKas_HR_jitter, xdata=baseKas_HR, ax0=ax0, ax1=ax1, alpha=0.5)
ax0.set_xlabel("Renormalized Baseline (ADC a.u.)")
ax0.set_ylabel("PH of events (a.u.)")
ax1.set_xlabel("Renormalized Baseline (ADC a.u.)")
ax1.set_ylabel("Corrected PH of events (a.u.)")
annotes = dataKas_HR.PH_ID
af =  AnnoteFinder(baseKas_HR,dataKas_HR_jitter, annotes, ax=ax0)
fig.canvas.mpl_connect('button_press_event', af)
fig.tight_layout()

print("\nMn Kb")
fig = plt.figure(figsize=(9, 4))
ax0 = fig.add_subplot(1,2,1)
ax1 = fig.add_subplot(1,2,2)
baseKb_HR = dataKb_HR.BSLN
dataKb_HR_jitter_bsln = applyCorr(ydata=dataKb_HR_jitter, xdata=baseKb_HR, ax0=ax0, ax1=ax1 , alpha=0.5)
ax0.set_xlabel("Renormalized Baseline (ADC a.u.)")
ax0.set_ylabel("PH of events (a.u.)")
ax1.set_xlabel("Renormalized Baseline (ADC a.u.)")
ax1.set_ylabel("Corrected PH of events (a.u.)")
annotes = dataKb_HR.PH_ID
af =  AnnoteFinder(baseKb_HR,dataKb_HR_jitter, annotes, ax=ax0)
fig.canvas.mpl_connect('button_press_event', af)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mn Kas
Fit Y=2.012e+02+ (-5.026e-02)*x**1+ (3.234e-06)*x**2
Fit Y corrected=5.896e+00+ (1.360e-12)*x**1+ (-8.750e-17)*x**2

Mn Kb


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fit Y=2.198e+02+ (-5.495e-02)*x**1+ (3.536e-06)*x**2
Fit Y corrected=6.367e+00+ (2.951e-12)*x**1+ (-1.898e-16)*x**2


## 4. Drift correction

In [31]:
plt.close()
fig = plt.figure(figsize=(10, 9))
ax1 = fig.add_subplot(4,2,1)
ax2 = fig.add_subplot(4,2,2)
ax3 = fig.add_subplot(4,2,3)
ax4 = fig.add_subplot(4,2,4)
ax5 = fig.add_subplot(4,2,5)
ax6 = fig.add_subplot(4,2,6)
ax7 = fig.add_subplot(4,2,7)
ax8 = fig.add_subplot(4,2,8)


time_break1 = 4250 + 1.57869e9
time_break2 = 5500 + 1.57869e9
#indices1 = np.argwhere(np.array(dataKas_HR.TIME)<time_break1)[:,0]
#indices2 = np.argwhere((np.array(dataKas_HR.TIME)>time_break1) & (np.array(dataKas_HR.TIME)<time_break2))[:,0]
#indices3 = np.argwhere(np.array(dataKas_HR.TIME)>time_break2)[:,0]
indices1 = np.argwhere(np.array(dataKas_noXT.TIME)<time_break1)[:,0]
indices2 = np.argwhere((np.array(dataKas_noXT.TIME)>time_break1) & (np.array(dataKas_noXT.TIME)<time_break2))[:,0]
indices3 = np.argwhere(np.array(dataKas_noXT.TIME)>time_break2)[:,0]

dataKas_HR_jitter_bsln_drift = np.zeros(len(dataKas_HR_jitter_bsln))
n1 = len(indices1)
n2 = len(indices2)
n3 = len(indices3)

#idxsort = np.argsort(timeKas_HR)
#ax0.scatter(np.arange(len(idxsort)), dataKas_HR.SIGNAL[idxsort], marker=".", s=5)
#ax1.scatter(dataKas_HR.TIME, dataKas_HR.SIGNAL,marker=".", s=5, color='black')
ax1.scatter(dataKas_noXT.TIME, dataKas_noXT.SIGNAL,marker=".", s=5, color='black')
ax1.set_xlabel("UTime")
ax1.set_ylabel("PH of events (a.u.)")
ax1.set_title("Raw PH")
ax2.scatter(dataKas_noXT.TIME, dataKas_HR_jitter_bsln,marker=".", s=5, color='tab:green')
ax2.set_xlabel("UTime")
ax2.set_ylabel("PH of events (a.u.)")
ax2.set_title("Jitter+BSLN corrected PH")


#dataKas_HR_jitter_bsln_drift[0:n1] = applyCorr(ydata=dataKas_HR_jitter_bsln[indices1], deg=1, xdata=dataKas_HR.TIME[indices1],
#                                          ax0=ax3, ax1=ax4, alpha=1)
dataKas_HR_jitter_bsln_drift[0:n1] = applyCorr(ydata=dataKas_HR_jitter_bsln[dataKas_noXT.TIME<time_break1], deg=1, 
                                               xdata=dataKas_noXT[dataKas_noXT.TIME<time_break1].TIME,
                                               ax0=ax3, ax1=ax4, alpha=1)
ax3.set_xlabel("UTime (time1)")
ax3.set_ylabel("PH of events (a.u.)")
ax3.set_title("J+B corr PH - time1")
ax4.set_xlabel("UTime (time1)")
ax4.set_ylabel("PH of events (a.u.)")
ax4.set_title("J+B+Drift Corr PH - time1")


#dataKas_HR_jitter_bsln_drift[n1:(n1+n2)] = applyCorr(ydata=dataKas_HR_jitter_bsln[indices2],deg=1, xdata=dataKas_HR.TIME[indices2],
#                                          ax0=ax5, ax1=ax6, alpha=1)
dataKas_HR_jitter_bsln_drift[n1:(n1+n2)] = applyCorr(ydata=dataKas_HR_jitter_bsln[(dataKas_noXT.TIME>time_break1) & (dataKas_noXT.TIME<time_break2)], 
                                                     xdata=dataKas_noXT[(dataKas_noXT.TIME>time_break1) & (dataKas_noXT.TIME<time_break2)].TIME,
                                                     deg=1, ax0=ax5, ax1=ax6, alpha=1)

ax5.set_xlabel("UTime (time2)")
ax5.set_ylabel("PH of events (a.u.)")
ax5.set_title("J+B corr PH - time2")
ax6.set_xlabel("UTime (time2)")
ax6.set_ylabel("Corrected PH of events (a.u.)")
ax6.set_title("J+B+Drift Corr PH - time2")

#dataKas_HR_jitter_bsln_drift[(n1+n2):(n1+n2+n3)] = applyCorr(ydata=dataKas_HR_jitter_bsln[indices3],deg=1, xdata=dataKas_HR.TIME[indices3],
#                                          ax0=ax7, ax1=ax8, alpha=1)
dataKas_HR_jitter_bsln_drift[(n1+n2):(n1+n2+n3)] = applyCorr(ydata=dataKas_HR_jitter_bsln[dataKas_noXT.TIME>time_break2], 
                                                             xdata=dataKas_noXT.TIME[dataKas_noXT.TIME>time_break2],
                                                             deg=1,ax0=ax7, ax1=ax8, alpha=1)    
ax7.set_xlabel("UTime (time3)")
ax7.set_ylabel("PH of events (a.u.)")
ax7.set_title("J+B corr PH - time3")
ax8.set_xlabel("UTime (time3)")
ax8.set_ylabel("Corrected PH of events (a.u.)")
ax8.set_title("J+B+Drift Corr PH - time3")

fig.tight_layout()
dataKb_HR_jitter_bsln_drift = np.copy(dataKb_HR_jitter_bsln)  # Kbeta no drift corrected

print(type(dataKas_HR_jitter))
print(type(dataKas_HR_jitter_bsln))
print(type(dataKas_HR_jitter_bsln_drift))

############## SO AS NOT TO USE DRIFT CORRECTION ###################################
dataKas_HR_jitter_bsln_drift = np.copy(dataKas_HR_jitter_bsln)  # Kas no drift corrected    !!!!!!!!!!!!!!!!!!!!

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fit Y=-7.230e+03+ (4.583e-06)*x**1
Fit Y corrected=5.898e+00+ (-2.763e-16)*x**1
Fit Y=3.525e+03+ (-2.229e-06)*x**1
Fit Y corrected=5.897e+00+ (6.982e-16)*x**1
Fit Y=-2.885e+01+ (2.201e-08)*x**1
Fit Y corrected=5.896e+00+ (4.356e-19)*x**1
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


## 4. Fit gaussians, create Gain scale and calibrate energies

In [33]:
# Pre-calibration histogram
plt.close()
nbinsKas = 100
nbinsKb = 50
centresA=[5888,5897] #[5700,5710] 
centresB=6368
(mean1bsln, mean2bsln, mean3bsln) = fit3gauss2hist(data1=1e3*dataKas_HR_jitter_bsln_drift, data2=1e3*dataKb_HR_jitter_bsln_drift,
                                        a1=0.06, a2=0.12, a3=0.1, sig1=5, sig2=5, sig3=5, 
                                        mean1=centresA[0], mean2=centresA[1], mean3=centresB,
                                        nbins1=nbinsKas, nbins2=nbinsKb, 
                                        xlab="reconstructed PH (a.u.)", plot=True, xsize=12,ysize=4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
# create gain scale Ka2, Ka1, Kb
plt.close()
calib_lines = tuple([float(x) for x in (lines)])
recon_lines = (mean1bsln,mean2bsln, mean3bsln)
coefs = gainScalePolyFit(xData=recon_lines, yData=calib_lines, deg=2, ylab="MnK Lines energies (eV)", xsize=10, ysize=4)
print("gain scale coefs=", coefs)

RMSE: 4.265908282671653e-12
R-squared: 1.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

gain scale coefs= [-1.78286085e+03  1.34663525e+00 -7.57394184e-06]


In [35]:
minEeV_Ka=5860
maxEeV_Ka=5920
#minEeV_Ka=5880
#maxEeV_Ka=5905

minEeV_Kb=6470
maxEeV_Kb=6505
# recalibrate energies
enerKas_HR_jitter_bsln_drift = np.zeros(len(dataKas_HR_jitter_bsln_drift))
enerKb_HR_jitter_bsln_drift = np.zeros(len(dataKb_HR_jitter_bsln_drift))

for i in range(len(coefs)):
    enerKas_HR_jitter_bsln_drift += coefs[i] * (1e3*dataKas_HR_jitter_bsln_drift)**(i)
enerKas_HR_jitter_bsln_drift = enerKas_HR_jitter_bsln_drift[(enerKas_HR_jitter_bsln_drift > minEeV_Ka) &
                                                            (enerKas_HR_jitter_bsln_drift < maxEeV_Ka)]

for i in range(len(coefs)):
    enerKb_HR_jitter_bsln_drift += coefs[i] * (1e3*dataKb_HR_jitter_bsln_drift)**(i)
enerKb_HR_jitter_bsln_drift = enerKb_HR_jitter_bsln_drift[(enerKb_HR_jitter_bsln_drift>minEeV_Kb) &
                                                          (enerKb_HR_jitter_bsln_drift<maxEeV_Kb)]
#print(enerKb_HR_jitter_bsln)

## 4 Fit histogram of baseline-jitter-corrected energies

#### 4.1 Test different number of bins and see how the residuals respond

In [36]:
%%script false --no-raise-error
# Plot residuals curve to select best number of bins for Kas and Kb
plt.clf()
fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

fitVoigt2hist(data=enerKas_HR_jitter_bsln_drift, lines=MnKas, nbins=np.arange(30,200,20), ax0=ax1)
#fitVoigt2hist(data=enerKas_HR_jitter_bsln, lines=MnKas, nbins=50, ax0=ax1)
ax1.set_title("MnKas")

fitVoigt2hist(data=enerKb_HR_jitter_bsln_drift, lines=MnKb, nbins=np.arange(20,100,10), ax0=ax2)
ax2.set_title("MnKb")

#### 4.2 Use the range of number of bins where residuals are quite estable and use them to get different FWHM; then take median value

In [37]:
#%%script false --no-raise-error
plt.close()
# Calculate mean 'fit' for range of acceptable number of bins in Kas range
#                                                               ===========
(ibmin, ibmax) = (80, 200)
fwhm_G = list()
err_fwhm_G = list()
nbins = list()
for nbinsKas in np.arange(ibmin, ibmax, 10):
    fw, efw, vv_fit = fitVoigt2hist(data=enerKas_HR_jitter_bsln_drift, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)    
    plt.clf()
    if fw < 0 or efw == 0:
        continue
    fwhm_G.append(fw)
    err_fwhm_G.append(efw)
    nbins.append(nbinsKas)
print("FWHM_Gs=", np.array2string(np.asarray(fwhm_G), formatter={'float_kind':lambda x: "%.3f" % x}))
median_fwhm_G = np.median(fwhm_G)
median_err_fwhm_G = np.sum(abs(np.asarray(fwhm_G)-median_fwhm_G))/len(fwhm_G)
median_index = min(range(len(fwhm_G)), key=lambda i: abs(fwhm_G[i]-median_fwhm_G))
nbinsKas = nbins[median_index]
print("median_index=", median_index)
print("FWHM_G[median_index]=",fwhm_G[median_index])
print("ERR_FWHM_G[median_index]=",err_fwhm_G[median_index])
print("nbinsKas[median_index]=", nbins[median_index])

print("Using ", len(fwhm_G), "different number of bins")
txt_Kas = '{:0.2f}'.format(median_fwhm_G) + "+/-" + '{:0.2f}'.format(max(median_err_fwhm_G, err_fwhm_G[median_index])) + "eV"
print("Median FWHM_G(Kas)=", txt_Kas)


FWHM_Gs= [3.011 2.963 2.995 2.952 2.941 2.941 2.945 2.944 2.964 2.951 2.935 2.947]
median_index= 9
FWHM_G[median_index]= 2.9510572529848997
ERR_FWHM_G[median_index]= 0.05749636236738238
nbinsKas[median_index]= 170
Using  12 different number of bins
Median FWHM_G(Kas)= 2.95+/-0.06eV


In [ ]:
#%%script false --no-raise-error

# Calculate mean 'fit' for range of acceptable number of bins in Kb range
#                                                               ===========
(ibmin, ibmax) = (40, 100)
fwhm_G = list()
err_fwhm_G = list()
nbins = list()
for nbinsKb in np.arange(ibmin, ibmax, 10):
    fw, efw, vv_fit = fitVoigt2hist(data=enerKb_HR_jitter_bsln_drift, lines=MnKb, nbins=int(nbinsKb), ax0=ax2)    
    plt.clf()
    if efw == 0 or fw < 0:
        continue
    fwhm_G.append(fw)
    err_fwhm_G.append(efw)
    nbins.append(nbinsKb)
print("FWHM_Gs=", np.array2string(np.asarray(fwhm_G), formatter={'float_kind':lambda x: "%.3f" % x}))
median_fwhm_G = np.median(fwhm_G)
median_err_fwhm_G = np.sum(abs(np.asarray(fwhm_G)-median_fwhm_G))/len(fwhm_G)
median_index = min(range(len(fwhm_G)), key=lambda i: abs(fwhm_G[i]-median_fwhm_G))
nbinsKb = nbins[median_index]
print("median_index=", median_index)
print("FWHM_G[median_index]=",fwhm_G[median_index])
print("ERR_FWHM_G[median_index]=",err_fwhm_G[median_index])
print("nbinsKb[median_index]=", nbins[median_index])

txt_Kb = '{:0.2f}'.format(median_fwhm_G) + "+/-" + '{:0.2f}'.format(max(median_err_fwhm_G, err_fwhm_G[median_index])) + "eV"
print("Median FWHM_G(Kb)=", txt_Kb)

In [ ]:
#%%script false --no-raise-error
plt.close()
# Plot representative (median) fitting 
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

print("Using nbinsKas=", nbinsKas, "from median estimation")
print("Using nbinsKb=", nbinsKb, "from median estimation")

nlines_Kas = MnKas.getNumber() 
nlines_Kb = MnKb.getNumber()
nlines = nlines_Kas + nlines_Kb 
lines_centres_vv = np.zeros(nlines)

fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKas_HR_jitter_bsln, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)
print(fwhm_G, err_fwhm_G)
for i in range(nlines_Kas):
     lines_centres_vv[i] = vvmod.param_sets[i*4][0]

fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKb_HR_jitter_bsln, lines=MnKb, nbins=int(nbinsKb), ax0=ax2)
print(fwhm_G, err_fwhm_G)
for i in range(nlines_Kb):
    j = i + nlines_Kas
    lines_centres_vv[j] = vvmod.param_sets[i*4][0]

x1 = min(enerKas_HR_jitter_bsln)
y1 = 20
ax1.text(x1,y1, "FWHM=" + txt_Kas + "\n Total counts=" + str(len(enerKas_HR_jitter_bsln)))
ax1.axvline(MnKas.energies_eV[0], ls="--", color="gray")

x2 = min(enerKb_HR_jitter_bsln)
y2 = 80
ax2.text(x2,y2, "FWHM=" + txt_Kb)

outfig=resDir + "/fit_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + "_jitter_bsln.png"
fig.savefig(outfig)
fig.tight_layout()

## 5. Recalibrate energies:
      - get new gain scale with fitted line centres
      - recalibrate energies
      - fit again to get FWHM more precisely

In [ ]:
# reverse au to energy transformation of lines centres using gain scale
#lines_centres_Kas 
# Ecal = coefs[0] + coefs[1] * PHau + coefs[2] * PHau^2 + ....
# to get roots:    0 = coefs[0]-Ecal + coefs[1] * PHau + coefs[2] * PHau^2 + ....
PHau = np.zeros(nlines)
newcoefs = np.zeros(len(coefs))
np.copyto(newcoefs, coefs)
print(coefs)
for i in range(nlines):  
    line = lines_centres_vv[i]
    #print("line=", line)
    coef0 = coefs[0] - line
    newcoefs[0] = coef0
    calpoly = P(newcoefs)
    roots = calpoly.roots()
    #print("Roots=", roots)
    PHau[i] = roots[np.abs(line-roots).argmin()]
    #PHau[i] = max(calpoly.roots())
    #if abs(line - PHau[i]) > 1000:
    #    PHau[i] = min(calpoly.roots())
    #if abs(line - PHau[i]) > 5000:
    #    raise RuntimeError("PH root in a.u. is too different from line Energy (eV)")
    
    #print(calpoly.roots())
    #print("PH for ", line, "=", PHau[i])

In [ ]:
# create NEW gain scale Ka2, Ka1, Kb
plt.close()
ph_lines = np.sort(PHau)
#print(ph_lines)
true_lines = np.sort(np.concatenate((MnKas.energies_eV, MnKb.energies_eV)))
#print(true_lines)
coefs_final = gainScalePolyFit(xData=ph_lines, yData=true_lines, deg=deg, ylab="MnK Lines energies (eV)")
print("gain scale coefs=", coefs_final)


In [ ]:
# FINAL recalibration of energies
enerKas_final = np.zeros(len(dataKas_HR_jitter_bsln))
enerKb_final = np.zeros(len(dataKb_HR_jitter_bsln))

for i in range(len(coefs_final)):
    enerKas_final += coefs_final[i] * (1e3*dataKas_HR_jitter_bsln)**(i)
    #enerKas_final = inter + slope * 1e3*dataKas_HR_jitter_bsln # eV
enerKas_final = enerKas_final[(enerKas_final > minEeV_Ka) & (enerKas_final < maxEeV_Ka)]

for i in range(len(coefs_final)):
    enerKb_final += coefs_final[i] * (1e3*dataKb_HR_jitter_bsln)**(i)
    #enerKb_final = inter + slope * 1e3*dataKb_HR_jitter_bsln # eV
enerKb_final = enerKb_final[(enerKb_final>minEeV_Kb) & (enerKb_final<maxEeV_Kb)]


In [ ]:
#%%script false --no-raise-error

# Calculate meadian 'fit' for range of acceptable number of bins in Kas range
#                                                               ===========
(ibmin, ibmax) = (80, 200)
fwhm_G = list()
err_fwhm_G = list()
nbins = list()
for nbinsKas in np.arange(ibmin, ibmax, 10):
    fw, efw, vv_fit = fitVoigt2hist(data=enerKas_final, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)    
    plt.clf()
    if fw < 0 or efw == 0:
        continue
    fwhm_G.append(fw)
    err_fwhm_G.append(efw)
    nbins.append(nbinsKas)
print("FWHM_Gs=", np.array2string(np.asarray(fwhm_G), formatter={'float_kind':lambda x: "%.3f" % x}))
median_fwhm_G = np.median(fwhm_G)
median_err_fwhm_G = np.sum(abs(np.asarray(fwhm_G)-median_fwhm_G))/len(fwhm_G)
median_index = min(range(len(fwhm_G)), key=lambda i: abs(fwhm_G[i]-median_fwhm_G))
nbinsKas = nbins[median_index]
print("median_index=", median_index)
print("FWHM_G[median_index]=",fwhm_G[median_index])
print("ERR_FWHM_G[median_index]=",err_fwhm_G[median_index])
print("nbinsKas[median_index]=", nbins[median_index])

print("Using ", len(fwhm_G), "different number of bins")
txt_Kas = '{:0.2f}'.format(median_fwhm_G) + "+/-" + '{:0.2f}'.format(max(median_err_fwhm_G, err_fwhm_G[median_index])) + "eV"
print("Median FWHM_G(Kas)=", txt_Kas)


In [ ]:
#%%script false --no-raise-error
plt.close()
# Plot representative (median) fitting 
#                    =====================
fig = plt.figure(figsize=(14, 6.5))
ax1 = fig.add_subplot(1,2,1)

fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKas_final, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)
print(fwhm_G, err_fwhm_G)
x1 = min(enerKas_final)
y1 = 20
ax1.text(x1,y1, "FWHM=" + txt_Kas + "\n Total counts=" + str(len(enerKas_final)))
ax1.axvline(MnKas.energies_eV[0], ls="--", color="gray")
ax1.set_title("Representative 'median' fitting")

outfig=resDir + "/fit_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + "_final.png"
fig.tight_layout()
fig.savefig(outfig)

In [ ]:
#%%script false --no-raise-error
# Plot representative fitting for a fixed size bin (only for Kas)
#                                   ================
plt.close()
fig = plt.figure(figsize=(13,6.5))
ax1 = fig.add_subplot(1,2,1)

nbinsKas = 300
fwhm_G, err_fwhm_G, vvmod = fitVoigt2hist(data=enerKas_final, lines=MnKas, nbins=int(nbinsKas), ax0=ax1)
print(fwhm_G, err_fwhm_G)

txt_Kas = '{:0.2f}'.format(fwhm_G) + "+/-" + '{:0.2f}'.format(err_fwhm_G) + "eV"
x1 = min(enerKas_HR_jitter_bsln)
y1 = 10
ax1.text(x1,y1, "FWHM=" + txt_Kas + "\n Total counts=" + str(len(enerKas_final)))
ax1.axvline(MnKas.energies_eV[0], ls="--", color="gray")
ax1.set_title("Plot representative -fixed bin- fitting")


outfig=resDir + "/fitKas_" + "pL" + str(plen) + "_" + method + str(oflen) + pBstr + "_final_300bins.png"
fig.tight_layout()
#fig.savefig(outfig)

### 6. Plot ALL calibrated energies

In [ ]:
# Events energy estimation
plt.close()
ener_final = np.zeros(len(data_HR))

for i in range(len(coefs_final)):
    ener_final += coefs_final[i] * (1e3*data_HR.SIGNAL)**(i)

# Have a look to histogram of ALL reconstructed data
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(1, 1, 1)
npulses = len(data_HR.SIGNAL)
ax1.set_xlabel("Calibrated energy")
ax1.set_ylabel("# events")
ax1.set_title(("Distribution of ALL events (" + str(npulses) + ")"))
ax1.hist(ener_final, bins=4000, density=False, label="Histogram", alpha=0.5)
plotMainXlines(ax1, alp_l=0.3, alp_t=0.6)
fig.tight_layout()

### 7. Plot FWHM for different recontruction methods

In [ ]:
#%%script false --no-raise-error
# plot variation of FWHM for different record lengths and for OPTFILT and preBuffer

rlen      = (4096)
# 300 bins fit
#optf =     (7.12, 3.03, 2.80, 2.74, 2.85, 2.85)
#optf_err = (0.14, 0.07, 0.08, 0.06, 0.07, 0.07)
#pB =       (9.19, 2.86, 2.64, 2.57, 2.71, 2.78)
#pB_err =   (0.18, 0.07, 0.07, 0.07, 0.07, 0.08)

optf =     (3.30)
optf_err = (0.07)
pB =       (3.15)
pB_err =   (0.07)

fig = plt.figure(figsize=(12,9))
ax1 = fig.add_subplot(1, 1, 1)
ax1.errorbar(rlen, optf, yerr=optf_err, label="ADC OPTFILT", marker='o',capsize=3, ls="-", alpha=0.5)
ax1.errorbar(rlen, pB, yerr=pB_err, label="ADC preBuffer75 OPTFILT", marker='o',capsize=2, ls="-")
ax1.tick_params(axis='both', which='major', labelsize=14)
ax1.set_ylim(2.5,10)
ax1.set_xlabel("record length (samples)",fontsize='x-large')
ax1.set_ylabel("FWHM (eV)", fontsize='x-large')
ax1.legend()
#ax1.grid()

axins = inset_axes(ax1, width="70%", height=1.9, bbox_transform=ax1.transAxes, bbox_to_anchor=(0.4, 0.3, 0.5, 0.9), loc=4)
axins.errorbar(rlen, optf, yerr=optf_err, label="ADC OPTFILT", marker='o',capsize=3, ls="-", alpha=0.5)
axins.errorbar(rlen, pB, yerr=pB_err, label="ADC preBuffer75 OPTFILT", marker='o',capsize=2, ls="-")
axins.set_ylim(2.5,4)
axins.grid()
axins.set_xlabel("record length (samples)",fontsize='large')
axins.set_ylabel("FWHM (eV)", fontsize='large')

outfig=resDir + "/fitKas_comparative_final_300bins.png"
fig.savefig(outfig)


In [ ]:
# set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_ylim([-4, 4])
ax.grid(True)
 
# generate x values
x = np.linspace(0, 2 * np.pi, 100)
 
 
def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))
 
 
@widgets.interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
    [l.remove() for l in ax.lines]
    ax.plot(x, my_sine(x, w, amp, phi), color='C0')